In [2]:
import pandas as pd
import endpoints
from datetime import datetime, timedelta
# from config import SITE_EMAIL, SITE_PASSWORD
import time
import numpy as np

In [9]:
import json

In [10]:
from urllib.parse import quote

In [3]:
today = datetime.today()
yesterday = today - timedelta(days=1)
today_str = today.strftime('%Y-%m-%d')
yesterday_str = yesterday.strftime('%Y-%m-%d')

In [4]:
EMAIL = "vnaumovich123@gmail.com"
PASSWORD = "6m78Hbgt"

In [5]:
cookies_dict = endpoints.get_cookies_local(EMAIL, PASSWORD)
session = endpoints.get_session()

In [ ]:

data = endpoints.get_l1_l2(yesterday_str, today_str, session, cookies_dict)

df = pd.json_normalize(data['data'])
df.rename({'id': 'l1_id', 'value': 'l1_name'}, axis=1, inplace=True)
df = df[['l1_id', 'l1_name', 'data']]\
    .explode('data')\
    .reset_index(drop=True)
df = df.drop(0)\
    .reset_index(drop=True)
df_expanded = pd.json_normalize(df['data'])
df = pd.concat([df[['l1_id', 'l1_name']], df_expanded], axis=1)\
    .rename({'id': 'l2_id', 'value': 'l2_name'}, axis=1)
df = df[['l1_id', 'l1_name', 'l2_id', 'l2_name']]

In [7]:
temp_dfs = []
for l2_id in df['l2_id']:
    data = endpoints.get_l3(yesterday_str, l2_id, today_str, session, cookies_dict)
    df_temp = pd.json_normalize(data['data'])
    df_temp['l2_id'] = l2_id
    temp_dfs.append(df_temp)
    time.sleep(1)

if temp_dfs:
    df_l3 = pd.concat(temp_dfs, ignore_index=True)
    df_l3 = df_l3.rename(columns={'id': 'l3_id', 'name': 'l3_name'})\
        .drop(['url', 'path', 'isFavorite'], axis=1)
else:
    df_l3 = pd.DataFrame()

In [ ]:
def get_wb_totals():
    # Параметры запроса
    category_id = "12413804913918531246"
    check_date = "2025-09-10"
    period_days = "30"

    # Формируем JSON query
    query_params = {
        "start": 0,
        "length": 0,
        "orderBy": "ordersSum",
        "orderDirection": "desc",
        "checkDate": check_date,
        "periodDays": period_days,
        "trendType": "day",
        "filters": {}
    }

    # Кодируем JSON для URL
    encoded_query = quote(json.dumps(query_params))

    # Формируем timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-0")

    # Собираем итоговый URL
    url = f"https://eggheads.solutions/analytics/wbCategory/getBrandsList/{category_id}.json?query={encoded_query}&dns-cache={timestamp}"

    return url

In [6]:
l3_id = '12413804913918531246'

In [11]:
import requests

In [14]:
response = requests.get(get_wb_totals(), cookies=cookies_dict)

In [15]:
response.text

'{"status":"ok","items":[],"totals":{"brandId":null,"lastRemains":null,"reviewsCount":null,"name":"Автотовары > OFFroad > Антипробуксовочное приспособление","isFavorite":false,"orders":755,"ordersSum":1523102,"loosesPercent":39.77,"avgSum":2017,"loosesSum":605759,"trend":[{"date":"2025-08-12T00:00:00+00:00","ordersSum":25652,"orders":14},{"date":"2025-08-13T00:00:00+00:00","ordersSum":25678,"orders":17},{"date":"2025-08-14T00:00:00+00:00","ordersSum":39771,"orders":22},{"date":"2025-08-15T00:00:00+00:00","ordersSum":19624,"orders":13},{"date":"2025-08-16T00:00:00+00:00","ordersSum":43742,"orders":27},{"date":"2025-08-17T00:00:00+00:00","ordersSum":28443,"orders":14},{"date":"2025-08-18T00:00:00+00:00","ordersSum":31479,"orders":17},{"date":"2025-08-19T00:00:00+00:00","ordersSum":32790,"orders":15},{"date":"2025-08-20T00:00:00+00:00","ordersSum":21538,"orders":13},{"date":"2025-08-21T00:00:00+00:00","ordersSum":27910,"orders":12},{"date":"2025-08-22T00:00:00+00:00","ordersSum":16288,"or

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df = pd.read_csv('l1_l2_data.csv')

In [31]:
# Заменяем строки с кавычками на NaN
df['l2_id'] = df['l2_id'].replace('', pd.NA, regex=False)

# Заполняем NaN значениями из l1_id
df['l2_id'] = df['l2_id'].fillna(df['l1_id'])